In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re

### Windowsでうまくできないため、GitBashで実施
IMDBをカレントフォルダにダウンロード
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
解凍
!tar zxf aclImdb_v1.tar.gz
aclImdb/train/unsupはラベル無しのため削除
!rm -rf aclImdb/train/unsup
IMDBデータセットの説明を表示
!cat aclImdb/README

大規模な映画レビューデータセットv1.0

概要

このデータセットには、映画レビューと関連するバイナリが含まれています
センチメント極性ラベル。のベンチマークとして機能することを目的としています
センチメント分類。このドキュメントでは、データセットがどのようになったかを概説します
収集され、提供されたファイルの使用方法。

データセット

コアデータセットには、2万5千のトレインに均等に分割された50,000件のレビューが含まれています
および25kテストセット。ラベルの全体的な分布はバランスが取れています（25k
posおよび25k neg）。また、追加の50,000ラベルなし
教師なし学習用のドキュメント。

コレクション全体では、30件を超えるレビューは許可されません
同じ映画のレビューには相関がある傾向があるため
評価。さらに、トレインとテストセットには、互いに素なセットが含まれています。
映画なので、暗記しても重要なパフォーマンスは得られません
映画固有の用語と、観察されるラベルに関連付けられた用語。の中に
ラベル付きのトレイン/テストセット、ネガティブレビューのスコアは10点中4点以下
肯定的なレビューのスコアは10点中7点以上です。
より中立的な評価は、トレイン/テストセットには含まれません。の中に
教師なしセット、あらゆる評価のレビューが含まれており、
偶数のレビュー> 5および<= 5。

ファイル

に対応する2つの最上位ディレクトリ[train /、test /]があります。
トレーニングおよびテストセット。それぞれには、[pos /、neg /]ディレクトリが含まれます。
バイナリラベルが正と負のレビュー。これらの中で
ディレクトリ、レビューは次の名前のテキストファイルに保存されます
規則[[id] _ [rating] .txt]ここで、[id]は一意のIDであり、[rating]は
そのレビューの星評価を1〜10のスケールで。たとえば、ファイル
[test / pos / 200_8.txt]は、ポジティブラベルのテストセットのテキストです。
IMDbの一意のID 200と星評価8/10の例。の
[train / unsup /]ディレクトリにはすべての評価に対して0があります。評価は
データセットのこの部分では省略されます。

また、レビューごとにIMDb URLを個別に含めます
[urls_ [pos、neg、unsup] .txt]ファイル。一意のID 200のレビューは
このファイルの200行目にURLがあります。常に変化するIMDbにより、
レビューに直接リンクすることはできませんが、映画の
レビューページ。

レビューテキストファイルに加えて、既にトークン化されたバッグが含まれています
私たちの実験で使用された単語（BoW）機能。これら
train / testディレクトリの.featファイルに保存されます。各.feat
ファイルはLIBSVM形式、ラベル付きのASCIIスパースベクトル形式
データ。これらのファイルの機能インデックスは0から始まり、テキスト
機能インデックスに対応するトークンは[imdb.vocab]にあります。だから
.featファイルの0：7の行は、[imdb.vocab]の最初の単語を意味します
（the）はそのレビューに7回登場します。

.featファイル形式の詳細については、LIBSVMページ：
http://www.csie.ntu.edu.tw/~cjlin/libsvm/

また、[imdbEr.txt]には、
（Potts、2011）によって計算された[imdb.vocab]の各トークン。期待される
評価は、単語の平均的な極性を理解するための良い方法です
データセット内。

データセットの引用

このデータセットを使用する場合、ACL 2011の論文を引用してください。
紹介します。このペーパーには、分類結果も含まれています。
あなたは比較したいかもしれません。


@InProceedings {maas-EtAl：2011：ACL-HLT2011、
  著者= {マース、アンドリューL.とデイリー、レイモンドE.とファム、ピーターT.とファン、ダンとNg、アンドリューY.とポッツ、クリストファー}、
  title = {感情分析のための学習単語ベクトル}、
  booktitle = {第49回計算言語学会の年次総会の議事録：Human Language Technologies}、
  月= {6月}、
  年= {2011}、
  住所= {米国オレゴン州ポートランド}、
  出版社= {計算言語学協会}、
  ページ= {142--150}、
  url = {http://www.aclweb.org/anthology/P11-1015}
}

参照資料

ポッツ、クリストファー。 2011.否定の否定性について。ナン・リーと
デイビッド・ルッツ編、意味論と言語理論の議事録20
636-659。

接触

質問/コメント/修正については、アンドリュー・マースまでご連絡ください
amaas@cs.stanford.edu

In [21]:
from sklearn.datasets import load_files

train_review = load_files('./aclImdb/train/', encoding='utf-8')
x_train, y_train = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
x_test, y_test = test_review.data, test_review.target

# ラベルの0,1と意味の対応の表示
print(train_review.target_names)

['neg', 'pos']


(25000,)

In [30]:
print("x_train : {}".format(x_train[2]))
display(len(x_train))
display(y_train.shape)
print("x_test : {}".format(x_test[0]))
display("y_test : {}".format(y_test.shape))

x_train : Everyone plays their part pretty well in this "little nice movie". Belushi gets the chance to live part of his life differently, but ends up realizing that what he had was going to be just as good or maybe even better. The movie shows us that we ought to take advantage of the opportunities we have, not the ones we do not or cannot have. If U can get this movie on video for around $10, it´d be an investment!


25000

(25000,)

x_test : Don't hate Heather Graham because she's beautiful, hate her because she's fun to watch in this movie. Like the hip clothing and funky surroundings, the actors in this flick work well together. Casey Affleck is hysterical and Heather Graham literally lights up the screen. The minor characters - Goran Visnjic {sigh} and Patricia Velazquez are as TALENTED as they are gorgeous. Congratulations Miramax & Director Lisa Krueger!


'y_test : (25000,)'

### 【問題1】BoWのスクラッチ実装 
以下の3文のBoWを求められるプログラムをscikit-learnを使わずに作成してください。1-gramと2-gramで計算してください。
This movie is SOOOO funny!!!
What a movie! I never
best movie ever!!!!! this movie

In [4]:
mini_dataset = \
  ["This movie is SOOOO funny!!!",
    "What a movie! I never",
    "best movie ever!!!!! this movie"]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
bow = (vectorizer.fit_transform(mini_dataset)).toarray()

# DataFrameにまとめる
df = pd.DataFrame(bow, columns=vectorizer.get_feature_names())
display(df)

,a,best,ever,funny,i,is,movie,never,soooo,this,what
0,0,0,0,1,0,1,1,0,1,1,0
1,1,0,0,0,1,0,1,1,0,0,1
2,0,1,1,0,0,0,2,0,0,1,0


In [6]:
class SrcBoW:
    def __init__(self):
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        
    def fit_transform(self, X):
        
        if self.token==1:
            X_list=[]
            for x in mini_dataset:
                X_list+=x.split(" ")
            X_list=set(X_list)
            
            X_last_list=[[0] * len(X_list)] * len(mini_dataset)
            for i,x1 in enumerate(mini_dataset):
                for x2 in x1.split(" "):
                    for j,x3 in enumerate(X_list):
                        if x2==x3:
                            X_last_list[i][j]+=1
        if self.token==2:
            X_list=[]
            for x in mini_dataset:
                X_list+=x.split(" ")
            X_list=set(X_list)
            
            X_last_list=[[0] * len(X_list)] * len(mini_dataset)
            for i,x1 in enumerate(mini_dataset):
                for x2 in x1.split(" "):
                    for j,x3 in enumerate(X_list):
                        if x2==x3:
                            X_last_list[i][j]+=1
        return X_last_list


In [7]:
src_bow=SrcBoW()
a=src_bow.fit_transform(mini_dataset)
a

[[1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1]]

In [8]:
X_list=[]
for x in mini_dataset:
    X_list+=x.split(" ")
X_list=set(X_list)
display(X_list)

{'I',
 'SOOOO',
 'This',
 'What',
 'a',
 'best',
 'ever!!!!!',
 'funny!!!',
 'is',
 'movie',
 'movie!',
 'never',
 'this'}

In [9]:
X_last_list=[[0] * len(X_list)] * len(mini_dataset)
for i,x1 in enumerate(mini_dataset):
    for x2 in x1.split(" "):
        for j,x3 in enumerate(X_list):
            if x2==x3:
                X_last_list[i][j]+=1
            
display(X_last_list)

[[1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1]]

In [44]:
X_list2=[]
X_list3=[]
X_list=[]
for i,x in enumerate(mini_dataset):
    x = re.sub("a-zA-Z0-9", "", x)
    n=0
    l=len(x.split(" "))
    for x2 in x.split(" "):
        n+=1
        l-=1
        if n==1:
            X_list2=x2.lower() 
        if n==2:
            X_list2=X_list2+ " "+x2.lower()             
            X_list.append(X_list2)
            n=0
        if l==0:
            X_list.append(x2.lower() )

display(X_list)

SyntaxError: invalid syntax (<ipython-input-44-ddd60d950cf9>, line 5)

In [38]:
X_last_list=[[0] * len(X_list)] * len(mini_dataset)
for i,x1 in enumerate(mini_dataset):
    #x1 = re.sub(r"\a-zA-Z0-9", "", x1)
    for x2 in x1.split(" "):
        for j,x3 in enumerate(X_list):
            if x2==x3:
                X_last_list[i][j]+=1
            
display(X_last_list)

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

In [33]:
mini_dataset = re.sub(r"\W", "", mini_dataset)
mini_dataset

TypeError: expected string or bytes-like object

### 【問題2】TF-IDFの計算 
IMDB映画レビューデータセットをTF-IDFによりベクトル化してください。NLTKのストップワードを利用し、最大の語彙数は5000程度に設定してください。テキストクリーニングやステミングなどの前処理はこの問題では要求しません。
TF-IDFの計算にはscikit-learnの以下のどちらかのクラスを使用してください。
sklearn.feature_extraction.text.TfidfVectorizer — scikit-learn 0.21.3 documentation

sklearn.feature_extraction.text.TfidfTransformer — scikit-learn 0.21.3 documentation
なお、scikit-learnでは標準的な式とは異なる式が採用されています。

In [16]:
import nltk
stop_words = nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print("stop word : {}".format(stop_words)) # 'i', 'me', 'my', ...

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anai\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
stop word : ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'whe

In [5]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b', max_features = 5)
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

,a,best,ever,movie,this
0,0,0,0,1,1
1,1,0,0,1,0
2,0,1,1,2,1


In [52]:
tfidf = TfidfVectorizer(token_pattern=r'\b\w+\b', stop_words=stop_words, max_features = 5000)
x = tfidf.fit_transform(x_train)
#display(x)
df_X_train = pd.DataFrame(x.toarray(), columns=tfidf.get_feature_names())
display(df_X_train.head(10))

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2014662 stored elements in Compressed Sparse Row format>

,0,00,000,1,10,100,11,12,13,13th,...,york,young,younger,youth,z,zero,zizek,zombie,zombies,zone
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.164505,0.0,0.0,0.0,0.136932,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.124442,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.085006,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.035996,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


### 【問題3】TF-IDFを用いた学習 
問題2で求めたベクトルを用いてIMDB映画レビューデータセットの学習・推定を行なってください。
モデルは2値分類が行える任意のものを利用してください。
ここでは精度の高さは求めませんが、最大の語彙数やストップワード、n-gramの数を変化させて影響を検証してみてください。

In [55]:
vocab = tfidf.vocabulary_
#display(list(vocab.keys()))
tfidf2 = TfidfVectorizer(vocabulary=list(vocab.keys()))
X_test = tfidf2.fit_transform(x_test)
df_X_test = pd.DataFrame(X_test.toarray(), columns=tfidf.get_feature_names())
display(df_X_test.head(10))

,0,00,000,1,10,100,11,12,13,13th,...,york,young,younger,youth,z,zero,zizek,zombie,zombies,zone
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.066126,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.032718,0.000000,0.0,0.0,0.0,0.066381,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.065152,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.106678,0.000000,0.000000,0.0,0.0,0.0,0.089385,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.000000,0.029369,0.000000,0.0,0.0,0.0,0.059586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.000000,0.000000,0.071021,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.000000,0.055620,0.070033,0.0,0.0,0.0,0.056423,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

C:\Users\anai\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\anai\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\anai\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\anai\AppData\Roaming\Python\Python36\site-pack

In [58]:
"""
th.kerasで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
# データセットの読み込み
X_train = np.array(df_X_train)
X_test = np.array(df_X_test)

# ラベルを数値に変換
#y[y=='Iris-versicolor'] = 0
#y[y=='Iris-virginica'] = 1
#y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

input_data = tf.keras.layers.Input(shape=(n_input,))
x = tf.keras.layers.Dense(n_hidden1, activation=tf.nn.relu)(input_data)
x = tf.keras.layers.Dense(n_hidden2, activation=tf.nn.relu)(x)
output = tf.keras.layers.Dense(n_classes, activation=tf.nn.sigmoid)(x)

model = tf.keras.Model(inputs=input_data, outputs=output)

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=10,
                    verbose=2,
                    validation_split=0.2)

y_pred_proba = model.predict(X_test)[:, 0]

# 確率を0, 1に変換
y_pred = np.where(y_pred_proba >0.5, 1, 0)

#print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)
#結果がいらず、評価のみ行う場合はevaluateメソッドも便利です。
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0927 16:24:30.891554  1156 deprecation.py:506] From C:\Users\anai\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0927 16:24:31.079048  1156 deprecation.py:323] From C:\Users\anai\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5000)]            0         
_________________________________________________________________
dense (Dense)                (None, 50)                250050    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 255,251
Trainable params: 255,251
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 - 6s - loss: 0.3311 - acc: 0.8600 - val_loss: 0.2932 - val_acc: 0.8768
Epoch 2/10
20000/20000 - 5s - loss: 0.2029 - acc: 0.9169 - val_loss: 0.3042 - val_acc: 0.8740
Epoch 3/10
20000

### 【問題4】TF-IDFのスクラッチ実装 
以下の3文のTF-IDFを求められるプログラムをscikit-learnを使わずに作成してください。標準的な式と、scikit-learnの採用している式の2種類を作成してください。正規化は不要です。

### 【問題5】コーパスの前処理 
コーパスの前処理として、特殊文字（!など）やURLの除去、大文字の小文字化といったことを行なってください。また、単語（トークン）はリストで分割してください。

In [ ]:
train_review = load_files('./aclImdb/train/', encoding='utf-8')
x_train, y_train = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
x_test, y_test = test_review.data, test_review.target

In [ ]:
from bs4 import BeautifulSoup

def clean_text(text):
    replaced_text = '\n'.join(s.strip() for s in text.splitlines()[2:] if s != '')  # skip header by [2:]
    replaced_text = replaced_text.lower()
    replaced_text = re.sub(r'[【】]', ' ', replaced_text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    return replaced_text

def clean_url(html_text):
    clean_text = re.sub(r'http\S+', '', html_text)
    return clean_text